#### Import all needed

In [26]:
import json
import csv
from datetime import datetime

#### Get Interchange Info

In [27]:
with open('/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/interchangeInfo.json', 'rb') as fp:
    interchangeInfo = json.load(fp)

#### Get Buses List

In [28]:
buses_list = []
for k,v in interchangeInfo.iteritems():
    buses_list.extend(v['buses'])
buses_list = set(buses_list)

#### Get Service Details 

In [29]:
calendar_details = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/calendar.txt', 'r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            calendar_details[row[0]] = {'operating_days': [row[1], row[2], row[3], row[4], row[5], row[6], row[7]],
                                        'start_date': row[8], 'end_date': row[9]
                                       }
            

#### Get Stop Info 

In [30]:
stop_details = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/stops.txt', 'r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            stop_details[row[0]] = { 'name' : row[2], 'lat' : row[3], 'lon' : row[4], 'wheelchair' : row[7] }

#### Get Trip Stop List

In [35]:
trip_stop_info = {}
trip_start_end_time = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/stop_times.txt','r') as fp:
        reader = csv.reader(fp)
        for row in reader:
            if reader.line_num > 1:
                trip_stop_info[row[0]] = trip_stop_info.get(row[0], [])
                trip_stop_info[row[0]].append({'id' : row[3], 'arrival' : row[1], 'departure' : row[2], 
                                               'distance' : row[8],
                                               'name' : stop_details[row[3]]['name'], 
                                               'lat' : stop_details[row[3]]['lat'],
                                               'lon' : stop_details[row[3]]['lon'], 
                                               'wheelchair' : stop_details[row[3]]['wheelchair']})
                if row[4] == "1":
                    trip_start_end_time[row[0]] = {'start_time' : row[1]}
                else:
                    trip_start_end_time[row[0]]['end_time'] = row[2]

#### Get Route Name

In [36]:
route_info = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/routes.txt','r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        route_info[row[0]] = row[3]

#### buses = {trips: [{route_id: xxx, route_name: xxx, service_id: xxx, start_date: xxx, end_date: xxx , operating_days: [], time_start: xxx , time_end: xxx, total_time: xxx, stops: [id: xxx, lat: xxx, lon: xxx, dist: xxx]}] }

In [38]:
bus_trips = {}
route_id_list = set({})
trip_id_list = set({})
for bus in buses_list:
    bus_trips[bus] = {'trips': []}
    

with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/trips.txt','r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            bus = row[2].split('-')[1]
            trip_id = row[2]
            route_id = row[0]
            service_id = row[1]
            direction = row[5]
            head_sign = row[4]
            wheelchair = row[7]
            if bus in buses_list:
                trip_ids = map(lambda x: x['id'], bus_trips[bus]['trips'])
                if trip_id not in trip_ids:
                    bus_trips[bus]['trips'].append({'id' : trip_id, 
                                                    'route_id': route_id, 
                                                    'route_name' : route_info[route_id],
                                                    'service_id': service_id, 
                                                    'direction' : direction, 
                                                    'head_sign' : head_sign, 
                                                    'wheelchair': wheelchair,
                                                    'operating_days': calendar_details[service_id]['operating_days'],
                                                    'start_date' : datetime.strptime(calendar_details[service_id]['start_date'], '%Y%m%d').isoformat(),                                
                                                    'end_date' : datetime.strptime(calendar_details[service_id]['end_date'], '%Y%m%d').isoformat(),
                                                    'start_time' : trip_start_end_time[trip_id]['start_time'],
                                                    'end_time' : trip_start_end_time[trip_id]['end_time'],
                                                    'stops' : trip_stop_info[trip_id]
                                                   })
                    route_id_list.add(route_id)
                    trip_id_list.add(trip_id)

#### Write Bus Info to files

In [39]:
for bus, bus_info in bus_trips.iteritems():
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as fp:
        json.dump(bus_info, fp, indent=4)